# التكامل بين Kernel الدلالي وخادم OpenBnB MCP

يستعرض هذا الدفتر كيفية استخدام Kernel الدلالي مع خادم OpenBnB MCP الفعلي للبحث عن أماكن إقامة حقيقية على Airbnb باستخدام MCPStdioPlugin. للوصول إلى LLM، يتم استخدام Azure AI Foundry. لإعداد متغيرات البيئة الخاصة بك، يمكنك متابعة [درس الإعداد](/00-course-setup/README.md)


## استيراد الحزم المطلوبة


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## إنشاء اتصال مكون MCP

سنقوم بالاتصال بخادم [OpenBnB MCP](https://github.com/openbnb-org/mcp-server-airbnb) باستخدام MCPStdioPlugin. يوفر هذا الخادم وظيفة البحث في Airbnb من خلال حزمة @openbnb/mcp-server-airbnb.


## إنشاء العميل

في هذا المثال، سنستخدم Azure AI Foundry للوصول إلى LLM. تأكد من إعداد متغيرات البيئة بشكل صحيح.


## إعداد البيئة

قم بتكوين إعدادات Azure OpenAI. تأكد من تعيين متغيرات البيئة التالية:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## فهم تكامل OpenBnB MCP

يتصل هذا الدفتر بالخادم الحقيقي **OpenBnB MCP** الذي يوفر وظيفة البحث الفعلية في Airbnb.

### كيف يعمل:

1. **MCPStdioPlugin**: يستخدم الاتصال عبر الإدخال/الإخراج القياسي مع خادم MCP  
2. **حزمة NPM حقيقية**: يقوم بتنزيل وتشغيل `@openbnb/mcp-server-airbnb` عبر npx  
3. **بيانات مباشرة**: يعيد بيانات العقارات الفعلية من واجهات برمجة التطبيقات الخاصة بـ Airbnb  
4. **اكتشاف الوظائف**: يكتشف الوكيل تلقائيًا الوظائف المتاحة من خادم MCP  

### الوظائف المتاحة:

عادةً ما يوفر خادم OpenBnB MCP الوظائف التالية:  
- **search_listings** - البحث عن عقارات Airbnb حسب الموقع والمعايير  
- **get_listing_details** - الحصول على معلومات تفصيلية حول عقارات محددة  
- **check_availability** - التحقق من التوافر لتواريخ محددة  
- **get_reviews** - استرجاع المراجعات للعقارات  
- **get_host_info** - الحصول على معلومات حول مضيفي العقارات  

### المتطلبات الأساسية:

- **Node.js** مثبت على نظامك  
- **اتصال بالإنترنت** لتنزيل حزمة خادم MCP  
- **NPX** متاح (يأتي مع Node.js)  

### اختبار الاتصال:

يمكنك اختبار خادم MCP يدويًا عن طريق تشغيل:  
```bash
npx -y @openbnb/mcp-server-airbnb
```  

سيقوم هذا بتنزيل وتشغيل خادم OpenBnB MCP، والذي يتصل به Semantic Kernel بعد ذلك للحصول على بيانات Airbnb الفعلية.  


## تشغيل الوكيل مع خادم OpenBnB MCP

سنقوم الآن بتشغيل الوكيل الذكي الذي يتصل بخادم OpenBnB MCP للبحث عن أماكن إقامة حقيقية على Airbnb في ستوكهولم لشخصين بالغين وطفل واحد. يمكنك تعديل قائمة `user_inputs` لتغيير معايير البحث حسب رغبتك.


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# الملخص
تهانينا! لقد نجحت في بناء وكيل ذكاء اصطناعي يتكامل مع البحث عن أماكن الإقامة في العالم الحقيقي باستخدام بروتوكول سياق النموذج (MCP):

## التقنيات المستخدمة:
- Semantic Kernel - لبناء وكلاء ذكيين باستخدام Azure OpenAI
- Azure AI Foundry - لتوفير قدرات النماذج اللغوية الكبيرة وإكمال المحادثات
- MCP (بروتوكول سياق النموذج) - لتكامل الأدوات بشكل معياري
- OpenBnB MCP Server - لتوفير وظيفة البحث الحقيقي في Airbnb
- Node.js/NPX - لتشغيل خادم MCP الخارجي

## ما تعلمته:
- تكامل MCP: ربط وكلاء Semantic Kernel بخوادم MCP الخارجية
- الوصول إلى البيانات في الوقت الحقيقي: البحث عن عقارات Airbnb الفعلية عبر واجهات برمجة التطبيقات الحية
- التواصل عبر البروتوكول: استخدام التواصل عبر stdio بين الوكيل وخادم MCP
- اكتشاف الوظائف: اكتشاف الوظائف المتاحة تلقائيًا من خوادم MCP
- استجابة متدفقة: التقاط وتسجيل استدعاءات الوظائف في الوقت الحقيقي
- عرض HTML: تنسيق استجابات الوكيل باستخدام جداول منسقة وعروض تفاعلية

## الخطوات التالية:
- دمج خوادم MCP إضافية (الطقس، الرحلات، المطاعم)
- بناء نظام متعدد الوكلاء يجمع بين بروتوكولي MCP و A2A
- إنشاء خوادم MCP مخصصة لمصادر البيانات الخاصة بك
- تنفيذ ذاكرة محادثة مستمرة عبر الجلسات
- نشر الوكيل على Azure Functions مع تنسيق خادم MCP
- إضافة قدرات المصادقة وحجز المستخدم



---

**إخلاء المسؤولية**:  
تمت ترجمة هذا المستند باستخدام خدمة الترجمة الآلية [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو معلومات غير دقيقة. يجب اعتبار المستند الأصلي بلغته الأصلية هو المصدر الموثوق. للحصول على معلومات حساسة أو هامة، يُوصى بالاستعانة بترجمة بشرية احترافية. نحن غير مسؤولين عن أي سوء فهم أو تفسيرات خاطئة ناتجة عن استخدام هذه الترجمة.
